In [2]:
import os
import glob
import re
import codecs

In [3]:
boost_distribution_directory = '/Users/bortolonmatteo/Downloads/boost_1_70_0'
boost_libs_directory = os.path.join(boost_distribution_directory, 'libs')
boost_headers_directory = os.path.join(boost_distribution_directory, 'boost')

In [4]:
def read_dependency(file_name, current_package):
    f=open(file_name, "r", encoding="latin_1")
    if f.mode == 'r':
        contents = f.read()
        includes = re.findall(r'#include <boost/.*>', contents)
        f.close()
        dependencies = set()
        for include in includes:
            substring = include[16:]
            point_find = substring.find(".")
            slash_find = substring.find("/")
            package_end_point = -1
            if point_find != -1 and slash_find != -1:
                package_end_point = min(point_find, slash_find)
            elif point_find != -1:
                package_end_point = point_find
            elif slash_find != -1:
                package_end_point = slash_find
            if package_end_point != -1:
                dependencies.add(substring[:package_end_point])
        dependencies.discard(current_package)
        return dependencies

In [5]:
libs = []
for f in glob.glob(boost_libs_directory + "/*", recursive=False):
    if not os.path.isdir(f):
        continue
    srcs_directory = os.path.join(f,'src')
    libDictionary = {
        'name': os.path.basename(f),
        'libsPath': f,
        'compileLibrary': os.path.exists(srcs_directory),
        'dependencies': set()
    }
    if libDictionary['compileLibrary']:
        srcs_files = []
        for src in glob.glob(srcs_directory + "/**/*.cpp", recursive=True):
            libDictionary['dependencies'] = libDictionary['dependencies'].union(read_dependency(src, libDictionary['name']))
            srcs_files.append(src.replace(boost_distribution_directory + '/', ''))
        libDictionary['srcs'] = srcs_files
    headers_directory = os.path.join(boost_headers_directory, libDictionary['name'])
    header_files = []
    for header in glob.glob(headers_directory + "/**/*.hpp", recursive=True):
            libDictionary['dependencies'] = libDictionary['dependencies'].union(read_dependency(header, libDictionary['name']))
            header_files.append(header.replace(boost_distribution_directory + '/', ''))
    if os.path.exists(headers_directory + ".hpp"):
        libDictionary['dependencies'] = libDictionary['dependencies'].union(read_dependency(headers_directory + ".hpp", libDictionary['name']))
        header_files.append((headers_directory + ".hpp").replace(boost_distribution_directory + '/', ''))
    libDictionary['hdrs'] = header_files
    
    libs.append(libDictionary)

In [6]:
libs

[{'name': 'compatibility',
  'libsPath': '/Users/bortolonmatteo/Downloads/boost_1_70_0/libs/compatibility',
  'compileLibrary': False,
  'dependencies': set(),
  'hdrs': []},
 {'name': 'fusion',
  'libsPath': '/Users/bortolonmatteo/Downloads/boost_1_70_0/libs/fusion',
  'compileLibrary': False,
  'dependencies': {'call_traits',
   'config',
   'core',
   'detail',
   'function_types',
   'functional',
   'get_pointer',
   'mpl',
   'preprocessor',
   'ref',
   'static_assert',
   'tuple',
   'type_traits',
   'typeof',
   'utility'},
  'hdrs': ['boost/fusion/container.hpp',
   'boost/fusion/support.hpp',
   'boost/fusion/mpl.hpp',
   'boost/fusion/sequence.hpp',
   'boost/fusion/algorithm.hpp',
   'boost/fusion/tuple.hpp',
   'boost/fusion/functional.hpp',
   'boost/fusion/view.hpp',
   'boost/fusion/adapted.hpp',
   'boost/fusion/iterator.hpp',
   'boost/fusion/tuple/make_tuple.hpp',
   'boost/fusion/tuple/tuple.hpp',
   'boost/fusion/tuple/tuple_fwd.hpp',
   'boost/fusion/tuple/tuple

In [9]:
contents = ""
for lib in libs:
    headers_string = '"' + '","'.join(lib['hdrs']) + '"'
    if lib['compileLibrary']:
        headers_string = "hdrs = [" + headers_string + "],"
        srcs_string = '"' + '","'.join(lib['srcs']) + '"'
        headers_string = headers_string + "\nsrcs = [" + srcs_string + "]"
    else:
        headers_string = "textual_hdrs = [" + headers_string + "]"
    deps = '":' + '",":'.join(libDictionary['dependencies']) + '"'
    contents += """
        cc_library(
          name = "{0}",
          {1},
          visibility = ["//visibility:public"],
          deps = [{2}],
        )""".format(lib['name'], headers_string, deps)
print(contents)


        cc_library(
          name = "compatibility",
          textual_hdrs = [""],
          visibility = ["//visibility:public"],
          deps = [":container_hash",":config",":cstdint",":aligned_storage",":core",":type_traits",":swap",":detail",":static_assert",":preprocessor",":throw_exception"],
        )
        cc_library(
          name = "fusion",
          textual_hdrs = ["boost/fusion/container.hpp","boost/fusion/support.hpp","boost/fusion/mpl.hpp","boost/fusion/sequence.hpp","boost/fusion/algorithm.hpp","boost/fusion/tuple.hpp","boost/fusion/functional.hpp","boost/fusion/view.hpp","boost/fusion/adapted.hpp","boost/fusion/iterator.hpp","boost/fusion/tuple/make_tuple.hpp","boost/fusion/tuple/tuple.hpp","boost/fusion/tuple/tuple_fwd.hpp","boost/fusion/tuple/tuple_tie.hpp","boost/fusion/tuple/detail/make_tuple.hpp","boost/fusion/tuple/detail/tuple.hpp","boost/fusion/tuple/detail/tuple_expand.hpp","boost/fusion/tuple/detail/tuple_fwd.hpp","boost/fusion/tuple/detail/tuple_tie.